In [26]:
#Setting up the environment

!java -version

#Install Spark
#download file
!wget -q http://apache.osuosl.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
#extract the file
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
#install findspark package
!pip install -q findspark

openjdk version "11.0.15" 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.20.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.20.04.1, mixed mode, sharing)


In [30]:
import os
pathToSpark = "/mnt/c/Users/walid/Desktop/Big Data/Final/spark-3.2.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = pathToSpark

In [29]:
import findspark
findspark.init()

# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("FinalProject").setMaster("local[*]")
sc=SparkContext(conf = conf)
spark = SparkSession(sparkContext=sc)

In [ ]:
Filepath = 

In [ ]:
"""
    shows most common playlist descriptions in the MPD

    Usage:

        python descriptions.py path-to-mpd-data
"""
import sys
import json
import re
import collections
import os

quick = False
max_files_for_quick_processing = 50
descriptions = collections.Counter()
ndescriptions = collections.Counter()


def process_mpd(path):
    count = 0
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            process_info(mpd_slice["info"])
            for playlist in mpd_slice["playlists"]:
                process_playlist(playlist)
            count += 1

            if quick and count > max_files_for_quick_processing:
                break

    show_summary()


def show_summary():
    print("descriptions")
    for k, v in descriptions.most_common():
        print(v, k)

    print()
    print("normalized descriptions")
    for k, v in ndescriptions.most_common():
        print(v, k)


def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", " ", name)
    name = re.sub(r"\s+", " ", name).strip()
    return name


def process_playlist(playlist):
    if "description" in playlist:
        desc = playlist["description"]
        ndesc = normalize_name(desc)
        descriptions[desc] += 1
        ndescriptions[ndesc] += 1


def process_info(_):
    pass


if __name__ == "__main__":
    path = sys.argv[1]
    if len(sys.argv) > 2 and sys.argv[2] == "--quick":
        quick = True
    process_mpd(path)
